In [20]:
import os, numpy as np, matplotlib as mpl

In [1]:
import mantid

In [2]:
from mantid import simpleapi as msa

# DGS reduction
Make sure to not do beam normalization, SofPhiEIsDistribution=0, and no vanadium (UseProcessedDetVan=0)

In [5]:
ls /SNS/SEQ/IPTS-20833/

Data/  nexus/  shared/


In [7]:
workdir = '/SNS/SEQ/IPTS-20833/shared/red-07272018'
!mkdir -p {workdir}
%cd {workdir}

/SNS/snfs1/instruments/SEQ/IPTS-20833/shared/red-07272018


Runs: 147504-147863

Crystal orientation: 6.86, 6.86, 19.8   90 90 120
u 100
v -0.008 0.016 1

In [13]:
a=b=6.86
c=19.8
alpha = beta = 90.
gamma = 120.
u = [1,0,0]
v = [-0.008, 0.016, 1]

In [14]:
Q1_projection = "1,0,0"
Q2_projection = "0,1,0"
Q3_projection = "0,0,1"

In [17]:
emin = -0.5*25
emax = 0.95*25

In [21]:
for run in range(147504, 147863+1):
    fn = '/SNS/SEQ/IPTS-20833/nexus/SEQ_%s.nxs.h5' % run
    data = msa.LoadEventNexus(fn)
    monws = msa.LoadNexusMonitors(fn)
    Ei, T0 = msa.GetEiT0atSNS(monws)
    dgs,_=msa.DgsReduction(
        SampleInputWorkspace=data,
        IncidentEnergyGuess=Ei,
        TimeZeroGuess=T0,
        UseIncidentEnergyGuess=True,
        SampleInputMonitorWorkspace=data,
        IncidentBeamNormalisation='None',
        SofPhiEIsDistribution='0')
    dgs=msa.CropWorkspace(InputWorkspace=dgs, XMin = emin, XMax = emax)
    msa.SetUB(dgs,a=a,b=b,c=c,alpha=alpha,beta=beta,gamma=gamma,u=u,v=v)
    msa.SetGoniometer(dgs, Axis0="phi,0,1,0,1")
    minValues,maxValues=msa.ConvertToMDMinMaxLocal(InputWorkspace=dgs, 
                                               QDimensions='Q3D',
                                               dEAnalysisMode='Direct',
                                               Q3DFrames='HKL',
                                               QConversionScales='HKL',
                                               Uproj=Q1_projection,
                                               Vproj=Q2_projection,
                                               Wproj=Q3_projection)

    mdpart=msa.ConvertToMD(dgs,
                       QDimensions='Q3D',
                       dEAnalysisMode='Direct',
                       Q3DFrames="HKL",
                       QConversionScales="HKL",
                       MinValues=minValues,
                       MaxValues=maxValues,
                       UProj=Q1_projection,
                       VProj=Q2_projection,
                       WProj=Q3_projection)
    msa.SaveMD(mdpart, "%s/mde/mdpart-%s.nxs" % (os.path.abspath('.'), run))

In [23]:
ls mde/*.nxs | wc -l

360
